In [1]:
!python.exe -m pip install --upgrade pip
%pip install -q langchain-elasticsearch
%pip install -q sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [31]:
from uuid import uuid4
from langchain_core.documents import Document
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore
from langchain_community.retrievers import ElasticSearchBM25Retriever
from langchain.embeddings import HuggingFaceEmbeddings

In [32]:
with open(
    file="ТИУ Сущности База знаний.txt",
    mode='r',
    encoding='utf-8'
) as file:
    text = file.read()
    
len(text)

271386

In [33]:
import re


entity_pattern = re.compile(r'\(entity\|([^|]+)\|([^|]+)\|([^|]+)\)')

relationship_pattern = re.compile(r'\(relationship\|([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)\)')

In [34]:
entities = []
for match in entity_pattern.finditer(text):
    entity_name, entity_type, entity_description = match.groups()
    entities.append({
        "name": entity_name.strip(),
        "type": entity_type.strip(),
        "description": entity_description.strip()
    })
    
len(entities)    

1592

In [35]:
entities[100]

{'name': 'БЛИЗКИЙ_РОДСТВЕННИК',
 'type': 'КАТЕГОРИЯ_ЛИЦ',
 'description': 'родственник первой степени родства'}

In [8]:
%pip install -q tf-keras

Note: you may need to restart the kernel to use updated packages.


In [36]:
embeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': False},
)

In [37]:
documents = []
for entity in entities:
    document = Document(
        page_content=entity["name"],
        metadata={
            "type": entity["type"], 
            "description": entity["description"]
        }
    )
    documents.append(document)

In [38]:
batch_size = 100
batches = []

for i in range(0, len(documents), batch_size):
    batches.append(documents[i:i+batch_size])

print(f"Всего создано {len(batches)} батчей.")

Всего создано 16 батчей.


In [39]:
es = Elasticsearch(
    hosts="https://elasticsearch-pv2s-production.up.railway.app/",
    basic_auth=("elastic", "2qq0lvgc89lwh6z5jp2q9280dcneaf95")
)

In [40]:
es.indices.delete(index="nodes")

ObjectApiResponse({'acknowledged': True})

In [41]:
elastic_store = ElasticsearchStore(
    es_url="https://elasticsearch-pv2s-production.up.railway.app/",
    index_name="nodes_embeddings",
    embedding=embeddings,
    es_user="elastic",
    es_password="2qq0lvgc89lwh6z5jp2q9280dcneaf95",
)

In [42]:
for batch in batches:
    elastic_store.add_documents(batch)

In [47]:
docs = elastic_store.similarity_search("Какой проходной балл на Нефтегазовое дело")
docs

[Document(metadata={'type': 'РЕЗУЛЬТАТ', 'description': 'итоговая оценка поступающего'}, page_content='ИТОГОВЫЙ_КОНКУРСНЫЙ_БАЛЛ'),
 Document(metadata={'type': 'ПАРАМЕТР', 'description': 'Типы источников финансирования мест'}, page_content='ИсточникиФинансированияМест'),
 Document(metadata={'type': 'ПОДГОТОВКА', 'description': 'подготовка по темам ЕГЭ, включая сложные задачи'}, page_content='ПОДГОТОВКА_ПО_ЕГЭ'),
 Document(metadata={'type': 'ПОКАЗАТЕЛЬ', 'description': 'показатель, характеризующий количество баллов, необходимое для поступления'}, page_content='СРЕДНИЕ_ПРОХОДНЫЕ_БАЛЛЫ')]

In [51]:
es.indices.delete(index="nodes_documents")

ObjectApiResponse({'acknowledged': True})

In [52]:
bm25_retriever = ElasticSearchBM25Retriever(
    client=es, 
    index_name="nodes_documents"
)

In [55]:
for batch in batches:
    bm25_retriever.add_texts([document.page_content for document in batch])

In [57]:
docs = bm25_retriever.invoke("Как поступить в рамках контрольных цифр")
docs

[Document(metadata={}, page_content='ПОДПУНКТ_«В»'),
 Document(metadata={}, page_content='ПРИЁМ В УНИВЕРСИТЕТ'),
 Document(metadata={}, page_content='ПОДГОТОВКА К ПОСТУПЛЕНИЮ В ВУЗ'),
 Document(metadata={}, page_content='ПЕРЕЧЕНЬ НЕОБХОДИМЫХ ДОКУМЕНТОВ ДЛЯ ПОСТУПЛЕНИЯ В ТИУ'),
 Document(metadata={}, page_content='ФЕДЕРАЛЬНЫЕ ЗАКОНЫ "О ГОСУДАРСТВЕННОЙ ПОЛИТИКЕ РОССИЙСКОЙ ФЕДЕРАЦИИ В ОТНОШЕНИИ СООТЕЧЕСТВЕННИКОВ ЗА РУБЕЖОМ" И "ОБ ОБРАЗОВАНИИ В РОССИЙСКОЙ ФЕДЕРАЦИИ"'),
 Document(metadata={}, page_content='НОМИНАЦИЯ ЛУЧШАЯ ПОСТАНОВКА ФИЗКУЛЬТУРНО – СПОРТИВНОЙ РАБОТЫ В ОБРАЗОВАТЕЛЬНЫХ УЧРЕЖДЕНИЯХ ВЫСШЕГО ОБРАЗОВАНИЯ ГОРОДА ТЮМЕНИ')]